# 61. Ray Casting と座標系 - NeRFへの橋渡し

**難易度**: ★★★☆☆（中級）

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] カメラからレイを生成する原理を理解する
- [ ] ピクセル座標からワールド座標へのレイ方向を計算できる
- [ ] レイに沿った点のサンプリング方法を実装できる
- [ ] レイと基本的な形状との交差判定を実装できる
- [ ] NeRFにおけるレイベースのレンダリングの基礎を理解する

## 前提知識

- ピンホールカメラモデル（Notebook 51）
- 座標変換と同次座標（Notebook 53）
- カメラキャリブレーション（Notebook 54）

## 目次

1. [Ray Castingの概念](#1-ray-castingの概念)
2. [カメラからのレイ生成](#2-カメラからのレイ生成)
3. [レイの数学的表現](#3-レイの数学的表現)
4. [レイに沿った点サンプリング](#4-レイに沿った点サンプリング)
5. [レイと形状の交差判定](#5-レイと形状の交差判定)
6. [NeRFとの関連](#6-nerfとの関連)
7. [まとめとセルフチェック](#7-まとめとセルフチェック)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, Optional, List
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)
print("ライブラリの読み込み完了")

---

## 1. Ray Castingの概念

### 1.1 Ray Castingとは

**Ray Casting（レイキャスティング）** は、カメラから各ピクセルに対応するレイ（光線）を飛ばし、シーン内のオブジェクトとの交差を計算する技術です。

```
従来のレンダリング（ラスタライゼーション）:
  3D頂点 → 投影 → 2Dピクセル
  
Ray Casting（逆方向）:
  2Dピクセル → レイ生成 → 3D交差点
```

### 1.2 なぜRay Castingが重要か

1. **物理的に正確**: 光の伝播を直接シミュレート
2. **NeRFの基盤**: Neural Radiance Fieldsはレイベースのレンダリング
3. **ボリュームレンダリング**: 連続的な3D表現との相性が良い
4. **微分可能**: 深層学習との統合が容易

In [ ]:
# Ray Castingの概念図
fig = plt.figure(figsize=(12, 5))

# 左: 従来のラスタライゼーション
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_title('従来のレンダリング\n(3D→2D投影)', fontsize=12)

# カメラ
camera_pos = np.array([0, 0, 0])
ax1.scatter(*camera_pos, s=200, c='blue', marker='o', label='カメラ')

# 3Dオブジェクト（三角形）
triangle = np.array([
    [2, -1, 3],
    [2, 1, 3],
    [2, 0, 5]
])
ax1.plot_trisurf(triangle[:, 0], triangle[:, 1], triangle[:, 2], 
                 color='green', alpha=0.7)

# 投影線
for v in triangle:
    ax1.plot([camera_pos[0], v[0]], [camera_pos[1], v[1]], 
             [camera_pos[2], v[2]], 'g--', alpha=0.5)

# 画像平面
img_plane_x = 1
y_range = np.linspace(-1, 1, 2)
z_range = np.linspace(2, 5, 2)
Y, Z = np.meshgrid(y_range, z_range)
X = np.ones_like(Y) * img_plane_x
ax1.plot_surface(X, Y, Z, alpha=0.3, color='cyan')

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.legend()

# 右: Ray Casting
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_title('Ray Casting\n(ピクセル→レイ→3D)', fontsize=12)

# カメラ
ax2.scatter(*camera_pos, s=200, c='blue', marker='o', label='カメラ')

# 複数のレイ
ray_directions = [
    [1, -0.3, 0.5],
    [1, 0, 0.5],
    [1, 0.3, 0.5],
    [1, -0.3, 0.8],
    [1, 0, 0.8],
    [1, 0.3, 0.8],
]

for d in ray_directions:
    d = np.array(d)
    d = d / np.linalg.norm(d) * 5
    ax2.plot([0, d[0]], [0, d[1]], [0, d[2]], 'r-', alpha=0.6, linewidth=1)
    ax2.scatter(d[0], d[1], d[2], c='red', s=20)

# 球（交差対象）
u = np.linspace(0, 2 * np.pi, 30)
v = np.linspace(0, np.pi, 20)
sphere_center = np.array([3, 0, 2])
sphere_radius = 0.8
xs = sphere_center[0] + sphere_radius * np.outer(np.cos(u), np.sin(v))
ys = sphere_center[1] + sphere_radius * np.outer(np.sin(u), np.sin(v))
zs = sphere_center[2] + sphere_radius * np.outer(np.ones(np.size(u)), np.cos(v))
ax2.plot_surface(xs, ys, zs, color='green', alpha=0.5)

ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.legend()

plt.tight_layout()
plt.show()

---

## 2. カメラからのレイ生成

### 2.1 ピクセル座標系からカメラ座標系へ

ピンホールカメラモデルでは、3D点 $\mathbf{X}_c = (X_c, Y_c, Z_c)^T$ がピクセル $(u, v)$ に投影されます：

$$
\begin{pmatrix} u \\ v \\ 1 \end{pmatrix} = \frac{1}{Z_c} \mathbf{K} \begin{pmatrix} X_c \\ Y_c \\ Z_c \end{pmatrix}
$$

逆に、ピクセル $(u, v)$ からレイ方向を求めるには、この投影を逆変換します：

$$
\begin{pmatrix} X_c \\ Y_c \\ Z_c \end{pmatrix} = Z_c \cdot \mathbf{K}^{-1} \begin{pmatrix} u \\ v \\ 1 \end{pmatrix}
$$

### 2.2 レイの定義

レイは **原点（origin）** と **方向（direction）** で定義されます：

$$
\mathbf{r}(t) = \mathbf{o} + t \cdot \mathbf{d}, \quad t \geq 0
$$

- $\mathbf{o}$: レイの原点（カメラ位置）
- $\mathbf{d}$: レイの方向（正規化されたベクトル）
- $t$: レイに沿った距離パラメータ

In [ ]:
def get_ray_directions_camera(H: int, W: int, K: np.ndarray) -> np.ndarray:
    """
    カメラ座標系でのレイ方向を計算
    
    Parameters:
    -----------
    H, W : int
        画像の高さと幅
    K : np.ndarray, shape (3, 3)
        カメラ内部パラメータ行列
        
    Returns:
    --------
    directions : np.ndarray, shape (H, W, 3)
        各ピクセルに対応するレイ方向（カメラ座標系）
    """
    # ピクセル座標グリッドを生成
    # 各ピクセルの中心を使用（+0.5のオフセット）
    i, j = np.meshgrid(
        np.arange(W, dtype=np.float32) + 0.5,
        np.arange(H, dtype=np.float32) + 0.5,
        indexing='xy'
    )
    
    # 内部パラメータの取得
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]
    
    # ピクセル座標をカメラ座標に変換
    # (u - cx) / fx, (v - cy) / fy, 1 という正規化座標
    directions = np.stack([
        (i - cx) / fx,
        (j - cy) / fy,  # 注: OpenCV座標系ではY軸が下向き
        np.ones_like(i)
    ], axis=-1)
    
    # 正規化（単位ベクトル化）
    directions = directions / np.linalg.norm(directions, axis=-1, keepdims=True)
    
    return directions

# テスト
H, W = 480, 640
K = np.array([
    [500, 0, 320],
    [0, 500, 240],
    [0, 0, 1]
], dtype=np.float32)

directions = get_ray_directions_camera(H, W, K)
print(f"レイ方向の形状: {directions.shape}")
print(f"\n中心ピクセル (320, 240) のレイ方向:")
print(f"  {directions[240, 320]}")
print(f"  → 真正面を向いている（ほぼ [0, 0, 1]）")

print(f"\n左上ピクセル (0, 0) のレイ方向:")
print(f"  {directions[0, 0]}")

print(f"\n右下ピクセル ({W-1}, {H-1}) のレイ方向:")
print(f"  {directions[H-1, W-1]}")

In [ ]:
# レイ方向の可視化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# X成分
im0 = axes[0].imshow(directions[:, :, 0], cmap='RdBu')
axes[0].set_title('レイ方向 X成分\n（左右）')
axes[0].set_xlabel('u (ピクセル)')
axes[0].set_ylabel('v (ピクセル)')
plt.colorbar(im0, ax=axes[0])

# Y成分
im1 = axes[1].imshow(directions[:, :, 1], cmap='RdBu')
axes[1].set_title('レイ方向 Y成分\n（上下）')
axes[1].set_xlabel('u (ピクセル)')
axes[1].set_ylabel('v (ピクセル)')
plt.colorbar(im1, ax=axes[1])

# Z成分
im2 = axes[2].imshow(directions[:, :, 2], cmap='viridis')
axes[2].set_title('レイ方向 Z成分\n（奥行き方向）')
axes[2].set_xlabel('u (ピクセル)')
axes[2].set_ylabel('v (ピクセル)')
plt.colorbar(im2, ax=axes[2])

plt.tight_layout()
plt.show()

print("観察:")
print("- X成分: 左端で負、右端で正（左右方向）")
print("- Y成分: 上端で負、下端で正（上下方向）")
print("- Z成分: 中心で最大、周辺で小さい（正面方向の強さ）")

### 2.3 ワールド座標系でのレイ

カメラ座標系からワールド座標系へ変換するには、カメラの外部パラメータ（回転 $\mathbf{R}$、並進 $\mathbf{t}$）を使用します：

$$
\mathbf{d}_{world} = \mathbf{R}^T \mathbf{d}_{camera}
$$

$$
\mathbf{o}_{world} = -\mathbf{R}^T \mathbf{t} = \mathbf{C}
$$

ここで $\mathbf{C}$ はワールド座標系でのカメラ中心です。

In [ ]:
def get_rays(H: int, W: int, K: np.ndarray, 
             R: np.ndarray, t: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    ワールド座標系でのレイ原点と方向を計算
    
    Parameters:
    -----------
    H, W : int
        画像の高さと幅
    K : np.ndarray, shape (3, 3)
        カメラ内部パラメータ
    R : np.ndarray, shape (3, 3)
        カメラ回転行列（ワールド→カメラ）
    t : np.ndarray, shape (3,)
        カメラ並進ベクトル
        
    Returns:
    --------
    rays_o : np.ndarray, shape (H, W, 3)
        各ピクセルのレイ原点（全て同じカメラ中心）
    rays_d : np.ndarray, shape (H, W, 3)
        各ピクセルのレイ方向（ワールド座標系）
    """
    # カメラ座標系でのレイ方向
    directions_camera = get_ray_directions_camera(H, W, K)
    
    # ワールド座標系に変換
    # R^T を使用（カメラ→ワールドへの変換）
    rays_d = directions_camera @ R  # (H, W, 3) @ (3, 3) = (H, W, 3)
    
    # 正規化
    rays_d = rays_d / np.linalg.norm(rays_d, axis=-1, keepdims=True)
    
    # カメラ中心（ワールド座標系）
    camera_center = -R.T @ t
    
    # 全ピクセルに同じ原点
    rays_o = np.broadcast_to(camera_center, (H, W, 3)).copy()
    
    return rays_o, rays_d

# カメラパラメータの設定
# カメラはZ軸方向を向いて、(0, 0, -5)の位置にある
R = np.eye(3)  # 回転なし
t = np.array([0, 0, -5])  # カメラはZ=-5に位置

# レイの生成
rays_o, rays_d = get_rays(H, W, K, R, t)

print(f"レイ原点の形状: {rays_o.shape}")
print(f"レイ方向の形状: {rays_d.shape}")
print(f"\nカメラ中心（レイ原点）: {rays_o[0, 0]}")
print(f"中心ピクセルのレイ方向: {rays_d[H//2, W//2]}")

In [ ]:
# 3Dでレイを可視化
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# カメラ位置
camera_center = rays_o[0, 0]
ax.scatter(*camera_center, s=200, c='blue', marker='o', label='カメラ')

# サンプルレイを描画（間引いて表示）
step = 60
ray_length = 8
for i in range(0, H, step):
    for j in range(0, W, step):
        o = rays_o[i, j]
        d = rays_d[i, j]
        end = o + ray_length * d
        
        # 中心に近いレイは赤、周辺は青
        dist_from_center = np.sqrt((i - H/2)**2 + (j - W/2)**2)
        color = plt.cm.coolwarm(dist_from_center / (np.sqrt((H/2)**2 + (W/2)**2)))
        
        ax.plot([o[0], end[0]], [o[1], end[1]], [o[2], end[2]], 
                c=color, alpha=0.6, linewidth=0.8)

# 画像平面の表示
img_z = camera_center[2] + 1  # カメラから1単位前
img_corners = []
for (i, j) in [(0, 0), (0, W-1), (H-1, W-1), (H-1, 0), (0, 0)]:
    o = rays_o[i, j]
    d = rays_d[i, j]
    t_param = (img_z - o[2]) / d[2]
    point = o + t_param * d
    img_corners.append(point)

img_corners = np.array(img_corners)
ax.plot(img_corners[:, 0], img_corners[:, 1], img_corners[:, 2], 
        'g-', linewidth=2, label='画像平面')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('カメラからのレイ（ワールド座標系）')
ax.legend()

# 視点調整
ax.view_init(elev=20, azim=45)

plt.tight_layout()
plt.show()

---

## 3. レイの数学的表現

### 3.1 パラメトリック表現

レイは以下のパラメトリック方程式で表されます：

$$
\mathbf{r}(t) = \mathbf{o} + t \cdot \mathbf{d}
$$

| パラメータ | 意味 |
|-----------|------|
| $\mathbf{o}$ | 原点（3次元ベクトル） |
| $\mathbf{d}$ | 方向（単位ベクトル） |
| $t$ | 距離パラメータ（$t \geq 0$） |

### 3.2 Near/Far境界

実際のレンダリングでは、有効な距離範囲を設定します：

- **near**: 最小距離（カメラに近すぎる点を除外）
- **far**: 最大距離（遠すぎる点を除外）

$$
t_{near} \leq t \leq t_{far}
$$

In [ ]:
class Ray:
    """レイを表すクラス"""
    
    def __init__(self, origin: np.ndarray, direction: np.ndarray,
                 near: float = 0.0, far: float = np.inf):
        """
        Parameters:
        -----------
        origin : np.ndarray, shape (3,)
            レイの原点
        direction : np.ndarray, shape (3,)
            レイの方向（自動的に正規化される）
        near : float
            最小距離
        far : float
            最大距離
        """
        self.origin = np.array(origin, dtype=np.float64)
        self.direction = np.array(direction, dtype=np.float64)
        self.direction = self.direction / np.linalg.norm(self.direction)
        self.near = near
        self.far = far
    
    def at(self, t: float) -> np.ndarray:
        """パラメータtでのレイ上の点を返す"""
        return self.origin + t * self.direction
    
    def is_valid_t(self, t: float) -> bool:
        """tが有効範囲内かチェック"""
        return self.near <= t <= self.far
    
    def __repr__(self):
        return f"Ray(o={self.origin}, d={self.direction}, near={self.near}, far={self.far})"

# テスト
ray = Ray(
    origin=[0, 0, 0],
    direction=[1, 0, 1],
    near=0.1,
    far=10.0
)

print(f"レイ: {ray}")
print(f"\nt=0での点: {ray.at(0)}")
print(f"t=1での点: {ray.at(1)}")
print(f"t=5での点: {ray.at(5)}")
print(f"\nt=0.05は有効? {ray.is_valid_t(0.05)}")
print(f"t=5は有効? {ray.is_valid_t(5)}")
print(f"t=15は有効? {ray.is_valid_t(15)}")

---

## 4. レイに沿った点サンプリング

### 4.1 均一サンプリング

レイに沿って等間隔に点をサンプリングします：

$$
t_i = t_{near} + \frac{i}{N-1}(t_{far} - t_{near}), \quad i = 0, 1, ..., N-1
$$

### 4.2 層化サンプリング（Stratified Sampling）

NeRFでは、各区間内でランダムにサンプリングすることで、より滑らかな結果を得ます：

$$
t_i = t_{near} + \frac{i + \xi_i}{N}(t_{far} - t_{near}), \quad \xi_i \sim U(0, 1)
$$

In [ ]:
def sample_along_ray_uniform(ray: Ray, n_samples: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    レイに沿って均一にサンプリング
    
    Returns:
    --------
    t_vals : np.ndarray, shape (n_samples,)
        サンプリング位置のtパラメータ
    points : np.ndarray, shape (n_samples, 3)
        サンプリング点の3D座標
    """
    t_vals = np.linspace(ray.near, ray.far, n_samples)
    points = ray.origin[None, :] + t_vals[:, None] * ray.direction[None, :]
    return t_vals, points

def sample_along_ray_stratified(ray: Ray, n_samples: int, 
                                perturb: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    """
    レイに沿って層化サンプリング（NeRFスタイル）
    
    Parameters:
    -----------
    perturb : bool
        Trueの場合、各区間内でランダムにずらす
    """
    # 区間の境界を計算
    t_vals = np.linspace(ray.near, ray.far, n_samples + 1)
    
    if perturb:
        # 各区間の中点にランダムな摂動を加える
        mids = 0.5 * (t_vals[:-1] + t_vals[1:])
        upper = t_vals[1:]
        lower = t_vals[:-1]
        # 各区間内でランダムサンプリング
        t_rand = np.random.uniform(size=n_samples)
        t_vals = lower + (upper - lower) * t_rand
    else:
        # 各区間の中点を使用
        t_vals = 0.5 * (t_vals[:-1] + t_vals[1:])
    
    points = ray.origin[None, :] + t_vals[:, None] * ray.direction[None, :]
    return t_vals, points

# 比較
ray = Ray(origin=[0, 0, 0], direction=[0, 0, 1], near=1.0, far=5.0)
n_samples = 10

t_uniform, pts_uniform = sample_along_ray_uniform(ray, n_samples)
t_stratified, pts_stratified = sample_along_ray_stratified(ray, n_samples, perturb=True)

print("均一サンプリングのt値:")
print(t_uniform)
print("\n層化サンプリングのt値:")
print(t_stratified)

In [ ]:
# サンプリング方法の可視化
fig, axes = plt.subplots(2, 1, figsize=(12, 6))

ray = Ray(origin=[0, 0, 0], direction=[0, 0, 1], near=1.0, far=5.0)
n_samples = 16

# 複数回サンプリングして分布を見る
n_trials = 50

# 均一サンプリング
ax = axes[0]
t_uniform, _ = sample_along_ray_uniform(ray, n_samples)
for trial in range(n_trials):
    ax.scatter(t_uniform, [trial]*n_samples, c='blue', s=10, alpha=0.5)
ax.set_title('均一サンプリング（毎回同じ位置）')
ax.set_xlabel('t (レイパラメータ)')
ax.set_ylabel('試行回数')
ax.set_xlim(ray.near - 0.2, ray.far + 0.2)
ax.axvline(ray.near, color='gray', linestyle='--', label='near')
ax.axvline(ray.far, color='gray', linestyle='--', label='far')

# 層化サンプリング
ax = axes[1]
for trial in range(n_trials):
    t_stratified, _ = sample_along_ray_stratified(ray, n_samples, perturb=True)
    ax.scatter(t_stratified, [trial]*n_samples, c='red', s=10, alpha=0.5)
ax.set_title('層化サンプリング（各区間内でランダム）')
ax.set_xlabel('t (レイパラメータ)')
ax.set_ylabel('試行回数')
ax.set_xlim(ray.near - 0.2, ray.far + 0.2)
ax.axvline(ray.near, color='gray', linestyle='--')
ax.axvline(ray.far, color='gray', linestyle='--')

# 区間境界を表示
boundaries = np.linspace(ray.near, ray.far, n_samples + 1)
for b in boundaries:
    ax.axvline(b, color='green', linestyle=':', alpha=0.3)

plt.tight_layout()
plt.show()

print("観察:")
print("- 均一サンプリング: 常に同じ位置にサンプル点")
print("- 層化サンプリング: 各区間（緑の点線）内でランダムにサンプル")
print("- 層化サンプリングはエイリアシングを軽減し、滑らかな結果を生む")

### 4.3 階層的サンプリング（Hierarchical Sampling）

NeRFでは、2段階のサンプリングを行います：

1. **Coarse（粗い）サンプリング**: 全体を均等にカバー
2. **Fine（細かい）サンプリング**: 重要な領域に集中

重要度は、coarseサンプルの密度/色の重みに基づいて決定されます。

In [ ]:
def sample_pdf(bins: np.ndarray, weights: np.ndarray, 
               n_samples: int, det: bool = False) -> np.ndarray:
    """
    重み付き分布からサンプリング（NeRFの階層的サンプリング用）
    
    Parameters:
    -----------
    bins : np.ndarray, shape (n_bins + 1,)
        ビンの境界
    weights : np.ndarray, shape (n_bins,)
        各ビンの重み（確率密度）
    n_samples : int
        サンプル数
    det : bool
        True: 決定的サンプリング、False: 確率的サンプリング
    """
    # 重みを確率分布に変換
    weights = weights + 1e-5  # ゼロ除算防止
    pdf = weights / np.sum(weights)
    cdf = np.cumsum(pdf)
    cdf = np.concatenate([[0], cdf])  # CDF: [0, cumsum]
    
    # サンプリング
    if det:
        u = np.linspace(0, 1, n_samples)
    else:
        u = np.random.uniform(size=n_samples)
    
    # 逆CDF変換
    inds = np.searchsorted(cdf, u, side='right')
    inds = np.clip(inds - 1, 0, len(weights) - 1)
    
    # ビン内で線形補間
    below = bins[inds]
    above = bins[inds + 1]
    t = (u - cdf[inds]) / (cdf[inds + 1] - cdf[inds] + 1e-5)
    
    samples = below + t * (above - below)
    return samples

# 階層的サンプリングのデモ
ray = Ray(origin=[0, 0, 0], direction=[0, 0, 1], near=1.0, far=5.0)
n_coarse = 8
n_fine = 16

# Coarseサンプリング
t_coarse, _ = sample_along_ray_stratified(ray, n_coarse, perturb=False)

# 仮の重み（実際はネットワーク出力）
# ここでは中央付近に物体があると仮定
weights = np.exp(-0.5 * ((t_coarse - 3.0) / 0.5)**2)  # ガウシアン風の重み

# Fineサンプリング（重み付き）
bins = np.concatenate([[ray.near], t_coarse, [ray.far]])
t_fine = sample_pdf(bins[:-1], np.concatenate([[0], weights]), n_fine)

# 全サンプルを結合してソート
t_all = np.sort(np.concatenate([t_coarse, t_fine]))

print(f"Coarseサンプル数: {len(t_coarse)}")
print(f"Fineサンプル数: {len(t_fine)}")
print(f"合計サンプル数: {len(t_all)}")

In [ ]:
# 階層的サンプリングの可視化
fig, axes = plt.subplots(3, 1, figsize=(12, 8))

# Coarseサンプルと重み
ax = axes[0]
ax.bar(t_coarse, weights, width=0.15, alpha=0.7, color='blue', label='重み')
ax.scatter(t_coarse, np.zeros_like(t_coarse), c='blue', s=100, 
           marker='^', label='Coarseサンプル', zorder=5)
ax.set_title('Step 1: Coarseサンプリング → 重み計算')
ax.set_xlabel('t')
ax.set_ylabel('重み')
ax.legend()
ax.set_xlim(ray.near - 0.2, ray.far + 0.2)

# 重みに基づくPDF
ax = axes[1]
pdf = weights / np.sum(weights)
ax.bar(t_coarse, pdf, width=0.15, alpha=0.7, color='orange', label='PDF')
ax.scatter(t_fine, np.zeros_like(t_fine) - 0.02, c='red', s=80, 
           marker='v', label='Fineサンプル', zorder=5)
ax.set_title('Step 2: 重みに基づいてFineサンプリング')
ax.set_xlabel('t')
ax.set_ylabel('確率密度')
ax.legend()
ax.set_xlim(ray.near - 0.2, ray.far + 0.2)

# 最終的なサンプル分布
ax = axes[2]
ax.scatter(t_coarse, np.ones_like(t_coarse) * 0.3, c='blue', s=100, 
           marker='^', label='Coarse')
ax.scatter(t_fine, np.ones_like(t_fine) * 0.6, c='red', s=80, 
           marker='v', label='Fine')
ax.scatter(t_all, np.zeros_like(t_all), c='green', s=50, 
           marker='o', label='結合')

# 物体位置を示す
ax.axvspan(2.5, 3.5, alpha=0.2, color='gray', label='物体領域')

ax.set_title('Step 3: 全サンプルを結合')
ax.set_xlabel('t')
ax.set_yticks([])
ax.legend(loc='upper right')
ax.set_xlim(ray.near - 0.2, ray.far + 0.2)

plt.tight_layout()
plt.show()

print("観察:")
print("- Coarseサンプルは全体を均等にカバー")
print("- Fineサンプルは重みの高い領域（物体周辺）に集中")
print("- 結合により、重要な領域を高密度でサンプリング")

---

## 5. レイと形状の交差判定

### 5.1 レイと球の交差

中心 $\mathbf{c}$、半径 $r$ の球との交差を求めます。

球の方程式:
$$
\|\mathbf{p} - \mathbf{c}\|^2 = r^2
$$

レイを代入:
$$
\|\mathbf{o} + t\mathbf{d} - \mathbf{c}\|^2 = r^2
$$

展開して $t$ について解くと、二次方程式になります：
$$
at^2 + bt + c = 0
$$

ここで:
- $a = \mathbf{d} \cdot \mathbf{d} = 1$（正規化された方向）
- $b = 2\mathbf{d} \cdot (\mathbf{o} - \mathbf{c})$
- $c = \|\mathbf{o} - \mathbf{c}\|^2 - r^2$

In [ ]:
def ray_sphere_intersection(ray: Ray, center: np.ndarray, 
                            radius: float) -> Optional[Tuple[float, np.ndarray]]:
    """
    レイと球の交差判定
    
    Returns:
    --------
    (t, point) : 交差点がある場合
    None : 交差なし
    """
    oc = ray.origin - center
    
    # 二次方程式の係数
    a = np.dot(ray.direction, ray.direction)  # = 1 if normalized
    b = 2.0 * np.dot(oc, ray.direction)
    c = np.dot(oc, oc) - radius**2
    
    # 判別式
    discriminant = b**2 - 4*a*c
    
    if discriminant < 0:
        return None  # 交差なし
    
    # 2つの解
    sqrt_disc = np.sqrt(discriminant)
    t1 = (-b - sqrt_disc) / (2*a)
    t2 = (-b + sqrt_disc) / (2*a)
    
    # 有効な最小のtを選択
    if ray.is_valid_t(t1):
        t = t1
    elif ray.is_valid_t(t2):
        t = t2
    else:
        return None
    
    point = ray.at(t)
    return t, point

# テスト
ray = Ray(origin=[0, 0, 0], direction=[0, 0, 1], near=0.1, far=10.0)
sphere_center = np.array([0, 0, 5])
sphere_radius = 1.0

result = ray_sphere_intersection(ray, sphere_center, sphere_radius)
if result:
    t, point = result
    print(f"交差あり!")
    print(f"  t = {t:.4f}")
    print(f"  交差点 = {point}")
else:
    print("交差なし")

In [ ]:
# レイと球の交差を可視化
fig = plt.figure(figsize=(12, 5))

# 交差するケース
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_title('交差するケース')

ray1 = Ray([0, 0, 0], [0, 0, 1], near=0.1, far=10)
center1 = np.array([0, 0, 5])
radius1 = 1.0

# 球を描画
u = np.linspace(0, 2*np.pi, 30)
v = np.linspace(0, np.pi, 20)
x = center1[0] + radius1 * np.outer(np.cos(u), np.sin(v))
y = center1[1] + radius1 * np.outer(np.sin(u), np.sin(v))
z = center1[2] + radius1 * np.outer(np.ones(np.size(u)), np.cos(v))
ax1.plot_surface(x, y, z, alpha=0.3, color='blue')

# レイを描画
t_vals = np.linspace(0, 8, 50)
ray_points = ray1.origin + t_vals[:, None] * ray1.direction
ax1.plot(ray_points[:, 0], ray_points[:, 1], ray_points[:, 2], 'r-', linewidth=2)
ax1.scatter(*ray1.origin, s=100, c='red', marker='o', label='レイ原点')

# 交差点
result = ray_sphere_intersection(ray1, center1, radius1)
if result:
    t, point = result
    ax1.scatter(*point, s=200, c='green', marker='*', label=f'交差点 (t={t:.2f})')

ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.legend()

# 交差しないケース
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_title('交差しないケース')

ray2 = Ray([0, 0, 0], [0, 1, 1], near=0.1, far=10)  # 方向がずれている

ax2.plot_surface(x, y, z, alpha=0.3, color='blue')

ray_points2 = ray2.origin + t_vals[:, None] * ray2.direction
ax2.plot(ray_points2[:, 0], ray_points2[:, 1], ray_points2[:, 2], 'r-', linewidth=2)
ax2.scatter(*ray2.origin, s=100, c='red', marker='o', label='レイ原点')

result2 = ray_sphere_intersection(ray2, center1, radius1)
if result2:
    t, point = result2
    ax2.scatter(*point, s=200, c='green', marker='*', label='交差点')
else:
    ax2.set_title('交差しないケース（Miss）')

ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.legend()

plt.tight_layout()
plt.show()

### 5.2 レイと平面の交差

法線 $\mathbf{n}$、点 $\mathbf{p}_0$ を通る平面との交差：

$$
(\mathbf{r}(t) - \mathbf{p}_0) \cdot \mathbf{n} = 0
$$

$$
t = \frac{(\mathbf{p}_0 - \mathbf{o}) \cdot \mathbf{n}}{\mathbf{d} \cdot \mathbf{n}}
$$

In [ ]:
def ray_plane_intersection(ray: Ray, plane_point: np.ndarray, 
                           plane_normal: np.ndarray) -> Optional[Tuple[float, np.ndarray]]:
    """
    レイと平面の交差判定
    
    Parameters:
    -----------
    plane_point : np.ndarray
        平面上の1点
    plane_normal : np.ndarray
        平面の法線ベクトル
    """
    # 法線を正規化
    normal = plane_normal / np.linalg.norm(plane_normal)
    
    # レイ方向と法線の内積
    denom = np.dot(ray.direction, normal)
    
    # レイが平面と平行な場合
    if abs(denom) < 1e-8:
        return None
    
    t = np.dot(plane_point - ray.origin, normal) / denom
    
    if not ray.is_valid_t(t):
        return None
    
    point = ray.at(t)
    return t, point

# テスト
ray = Ray([0, 0, 0], [1, 1, 1], near=0.1, far=10)
plane_point = np.array([3, 3, 3])
plane_normal = np.array([1, 1, 1])  # 斜めの平面

result = ray_plane_intersection(ray, plane_point, plane_normal)
if result:
    t, point = result
    print(f"交差あり!")
    print(f"  t = {t:.4f}")
    print(f"  交差点 = {point}")

### 5.3 レイとAABB（軸平行境界ボックス）の交差

AABBは高速な交差判定に使用されます。Slab法を使います：

各軸について、レイがスラブ（2つの平行な平面）を通過する範囲 $[t_{min}, t_{max}]$ を計算し、全軸の交差を求めます。

In [ ]:
def ray_aabb_intersection(ray: Ray, bbox_min: np.ndarray, 
                          bbox_max: np.ndarray) -> Optional[Tuple[float, float]]:
    """
    レイとAABB（軸平行境界ボックス）の交差判定
    
    Returns:
    --------
    (t_enter, t_exit) : 交差する場合、入口と出口のt値
    None : 交差なし
    """
    t_min = ray.near
    t_max = ray.far
    
    for i in range(3):  # x, y, z 各軸
        if abs(ray.direction[i]) < 1e-8:
            # レイがこの軸に平行
            if ray.origin[i] < bbox_min[i] or ray.origin[i] > bbox_max[i]:
                return None  # ボックスの外
        else:
            # スラブとの交差
            inv_d = 1.0 / ray.direction[i]
            t1 = (bbox_min[i] - ray.origin[i]) * inv_d
            t2 = (bbox_max[i] - ray.origin[i]) * inv_d
            
            if t1 > t2:
                t1, t2 = t2, t1
            
            t_min = max(t_min, t1)
            t_max = min(t_max, t2)
            
            if t_min > t_max:
                return None
    
    return t_min, t_max

# テスト
ray = Ray([0, 0, 0], [1, 1, 1], near=0.1, far=20)
bbox_min = np.array([2, 2, 2])
bbox_max = np.array([4, 4, 4])

result = ray_aabb_intersection(ray, bbox_min, bbox_max)
if result:
    t_enter, t_exit = result
    print(f"AABBと交差!")
    print(f"  入口 t = {t_enter:.4f}, 点 = {ray.at(t_enter)}")
    print(f"  出口 t = {t_exit:.4f}, 点 = {ray.at(t_exit)}")

In [ ]:
# AABB交差の可視化
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# AABB（ワイヤーフレーム）
def draw_box(ax, bbox_min, bbox_max, color='blue', alpha=0.3):
    vertices = [
        [bbox_min[0], bbox_min[1], bbox_min[2]],
        [bbox_max[0], bbox_min[1], bbox_min[2]],
        [bbox_max[0], bbox_max[1], bbox_min[2]],
        [bbox_min[0], bbox_max[1], bbox_min[2]],
        [bbox_min[0], bbox_min[1], bbox_max[2]],
        [bbox_max[0], bbox_min[1], bbox_max[2]],
        [bbox_max[0], bbox_max[1], bbox_max[2]],
        [bbox_min[0], bbox_max[1], bbox_max[2]],
    ]
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # 底面
        [4, 5], [5, 6], [6, 7], [7, 4],  # 上面
        [0, 4], [1, 5], [2, 6], [3, 7],  # 側面
    ]
    for edge in edges:
        p1, p2 = vertices[edge[0]], vertices[edge[1]]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 
                color=color, linewidth=2)

draw_box(ax, bbox_min, bbox_max)

# 複数のレイをテスト
rays = [
    Ray([0, 0, 0], [1, 1, 1], near=0.1, far=20),      # 交差
    Ray([0, 0, 0], [1, 0, 0], near=0.1, far=20),      # ミス
    Ray([0, 3, 3], [1, 0, 0], near=0.1, far=20),      # 交差
]
colors = ['green', 'red', 'purple']

for ray, color in zip(rays, colors):
    # レイを描画
    t_vals = np.linspace(0, 8, 50)
    ray_points = ray.origin + t_vals[:, None] * ray.direction
    ax.plot(ray_points[:, 0], ray_points[:, 1], ray_points[:, 2], 
            f'{color[0]}--', linewidth=1.5, alpha=0.7)
    ax.scatter(*ray.origin, s=100, c=color, marker='o')
    
    # 交差判定
    result = ray_aabb_intersection(ray, bbox_min, bbox_max)
    if result:
        t_enter, t_exit = result
        enter_point = ray.at(t_enter)
        exit_point = ray.at(t_exit)
        ax.scatter(*enter_point, s=150, c=color, marker='>', zorder=10)
        ax.scatter(*exit_point, s=150, c=color, marker='<', zorder=10)
        # 交差部分を太線で
        ax.plot([enter_point[0], exit_point[0]], 
                [enter_point[1], exit_point[1]], 
                [enter_point[2], exit_point[2]], 
                color=color, linewidth=3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('レイとAABBの交差判定')

plt.tight_layout()
plt.show()

---

## 6. NeRFとの関連

### 6.1 NeRFにおけるレイの役割

**Neural Radiance Fields (NeRF)** では、レイベースのレンダリングが核心技術です：

1. **入力**: ピクセル座標 $(u, v)$ からレイを生成
2. **サンプリング**: レイに沿って点を階層的にサンプリング
3. **クエリ**: 各点でニューラルネットワークに (位置, 方向) を入力
4. **出力**: 色 $\mathbf{c}$ と密度 $\sigma$ を取得
5. **レンダリング**: ボリュームレンダリング方程式で色を統合

### 6.2 ボリュームレンダリング方程式（プレビュー）

$$
C(\mathbf{r}) = \int_{t_n}^{t_f} T(t) \sigma(\mathbf{r}(t)) \mathbf{c}(\mathbf{r}(t), \mathbf{d}) dt
$$

ここで:
- $T(t) = \exp\left(-\int_{t_n}^{t} \sigma(\mathbf{r}(s)) ds\right)$: 透過率
- $\sigma$: 密度
- $\mathbf{c}$: 色

**次のNotebook（62. ボリュームレンダリング）で詳しく学びます。**

In [ ]:
# NeRFスタイルのレイ処理デモ

def nerf_style_ray_processing(H: int, W: int, K: np.ndarray,
                              R: np.ndarray, t: np.ndarray,
                              near: float, far: float,
                              n_samples: int) -> dict:
    """
    NeRFスタイルのレイ生成とサンプリング
    
    Returns:
    --------
    dict: レイ情報を含む辞書
    """
    # レイ生成
    rays_o, rays_d = get_rays(H, W, K, R, t)
    
    # サンプリング（層化）
    t_vals = np.linspace(near, far, n_samples + 1)
    t_vals = 0.5 * (t_vals[:-1] + t_vals[1:])  # 中点
    
    # 摂動を追加
    mids = 0.5 * (t_vals[:-1] + t_vals[1:])
    upper = np.concatenate([mids, t_vals[-1:]])
    lower = np.concatenate([t_vals[:1], mids])
    t_rand = np.random.uniform(size=(H, W, n_samples))
    t_vals_perturbed = lower + (upper - lower) * t_rand
    
    # サンプル点の計算
    # points[i, j, k, :] = rays_o[i, j] + t_vals[k] * rays_d[i, j]
    points = rays_o[:, :, None, :] + t_vals_perturbed[:, :, :, None] * rays_d[:, :, None, :]
    
    return {
        'rays_o': rays_o,           # (H, W, 3)
        'rays_d': rays_d,           # (H, W, 3)
        't_vals': t_vals_perturbed, # (H, W, n_samples)
        'points': points,           # (H, W, n_samples, 3)
    }

# 小さい画像でデモ
H_demo, W_demo = 64, 64
K_demo = np.array([
    [50, 0, 32],
    [0, 50, 32],
    [0, 0, 1]
], dtype=np.float32)
R_demo = np.eye(3)
t_demo = np.array([0, 0, 0])

result = nerf_style_ray_processing(
    H_demo, W_demo, K_demo, R_demo, t_demo,
    near=2.0, far=6.0, n_samples=64
)

print("NeRFスタイルのレイ処理結果:")
print(f"  レイ原点: {result['rays_o'].shape}")
print(f"  レイ方向: {result['rays_d'].shape}")
print(f"  tパラメータ: {result['t_vals'].shape}")
print(f"  サンプル点: {result['points'].shape}")
print(f"\n合計クエリ点数: {H_demo * W_demo * 64:,}")

In [ ]:
# NeRFパイプラインの概念図
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# 1. レイ生成
ax = axes[0]
ax.set_title('Step 1: レイ生成', fontsize=12)
# 簡略化した図
ax.plot([0.2, 0.8], [0.5, 0.5], 'b-', linewidth=3)
ax.scatter([0.2], [0.5], s=200, c='blue', marker='o')
ax.annotate('カメラ', (0.2, 0.4), ha='center')
ax.annotate('(u, v)', (0.8, 0.55), ha='center')
for angle in np.linspace(-0.2, 0.2, 5):
    ax.plot([0.2, 0.8], [0.5, 0.5 + angle], 'r-', alpha=0.5, linewidth=1)
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.text(0.5, 0.1, r'$\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}$', 
        ha='center', fontsize=11)

# 2. サンプリング
ax = axes[1]
ax.set_title('Step 2: サンプリング', fontsize=12)
ax.plot([0.1, 0.9], [0.5, 0.5], 'r-', linewidth=2)
sample_x = np.linspace(0.2, 0.85, 8)
ax.scatter(sample_x, [0.5]*8, s=100, c='green', marker='o', zorder=5)
ax.annotate('near', (0.2, 0.4), ha='center')
ax.annotate('far', (0.85, 0.4), ha='center')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.text(0.5, 0.1, r'$\{\mathbf{x}_i\}_{i=1}^N$', ha='center', fontsize=11)

# 3. ネットワーククエリ
ax = axes[2]
ax.set_title('Step 3: NN クエリ', fontsize=12)
# MLPの図
layers = [3, 8, 8, 4]
x_positions = np.linspace(0.15, 0.85, len(layers))
for i, (x, n) in enumerate(zip(x_positions, layers)):
    y_positions = np.linspace(0.3, 0.7, n)
    ax.scatter([x]*n, y_positions, s=100, c='purple', marker='o')
    if i < len(layers) - 1:
        for y1 in y_positions:
            y_next = np.linspace(0.3, 0.7, layers[i+1])
            for y2 in y_next:
                ax.plot([x, x_positions[i+1]], [y1, y2], 'gray', 
                        alpha=0.2, linewidth=0.5)
ax.annotate(r'$(\mathbf{x}, \mathbf{d})$', (0.1, 0.5), ha='center')
ax.annotate(r'$(\mathbf{c}, \sigma)$', (0.92, 0.5), ha='left')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.text(0.5, 0.1, r'$F_\theta(\mathbf{x}, \mathbf{d}) \rightarrow (\mathbf{c}, \sigma)$', 
        ha='center', fontsize=11)

# 4. ボリュームレンダリング
ax = axes[3]
ax.set_title('Step 4: レンダリング', fontsize=12)
# グラデーション的な色の積分
colors = plt.cm.viridis(np.linspace(0.2, 0.8, 8))
x_pos = np.linspace(0.15, 0.7, 8)
for i, (x, c) in enumerate(zip(x_pos, colors)):
    alpha = 0.3 + 0.5 * (1 - i/8)
    ax.add_patch(plt.Rectangle((x, 0.35), 0.08, 0.3, 
                                facecolor=c, alpha=alpha))
ax.annotate(r'$\rightarrow$', (0.78, 0.5), fontsize=20)
ax.add_patch(plt.Rectangle((0.85, 0.4), 0.1, 0.2, 
                            facecolor='orange', edgecolor='black'))
ax.annotate('C', (0.9, 0.5), ha='center', fontsize=12, fontweight='bold')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')
ax.text(0.5, 0.1, r'$C = \sum_i T_i \alpha_i \mathbf{c}_i$', 
        ha='center', fontsize=11)

plt.tight_layout()
plt.show()

### 6.3 位置エンコーディング（Positional Encoding）

NeRFでは、座標をそのままMLPに入力するのではなく、高周波の位置エンコーディングを適用します：

$$
\gamma(p) = \left( \sin(2^0\pi p), \cos(2^0\pi p), ..., \sin(2^{L-1}\pi p), \cos(2^{L-1}\pi p) \right)
$$

これにより、ニューラルネットワークが高周波の詳細を学習できるようになります。

In [ ]:
def positional_encoding(x: np.ndarray, L: int = 10) -> np.ndarray:
    """
    NeRFスタイルの位置エンコーディング
    
    Parameters:
    -----------
    x : np.ndarray, shape (..., D)
        入力座標
    L : int
        周波数の数
        
    Returns:
    --------
    encoded : np.ndarray, shape (..., D * 2 * L)
        エンコードされた座標
    """
    encodings = []
    
    for i in range(L):
        freq = 2.0 ** i * np.pi
        encodings.append(np.sin(freq * x))
        encodings.append(np.cos(freq * x))
    
    return np.concatenate(encodings, axis=-1)

# デモ
x = np.array([0.5, 0.3, 0.7])  # 3D座標
encoded = positional_encoding(x, L=6)

print(f"入力次元: {x.shape[-1]}")
print(f"出力次元: {encoded.shape[-1]}")
print(f"\n入力: {x}")
print(f"\nエンコード後（最初の12成分）:")
print(encoded[:12])

In [ ]:
# 位置エンコーディングの可視化
fig, axes = plt.subplots(2, 1, figsize=(14, 6))

# 1Dの場合
x_1d = np.linspace(0, 1, 200)
L = 6

ax = axes[0]
ax.set_title(f'位置エンコーディング (L={L})', fontsize=12)

colors = plt.cm.viridis(np.linspace(0, 1, L))
for i in range(L):
    freq = 2.0 ** i * np.pi
    ax.plot(x_1d, np.sin(freq * x_1d), color=colors[i], 
            label=f'sin(2^{i}πx)', linewidth=1.5, alpha=0.8)

ax.set_xlabel('x')
ax.set_ylabel('sin(2^i πx)')
ax.legend(loc='upper right', ncol=3)
ax.grid(True, alpha=0.3)

# エンコード後の「指紋」
ax = axes[1]
encoded_1d = positional_encoding(x_1d[:, None], L=L)
im = ax.imshow(encoded_1d.T, aspect='auto', cmap='RdBu',
               extent=[0, 1, 0, encoded_1d.shape[1]])
ax.set_title('位置エンコーディング（ヒートマップ）', fontsize=12)
ax.set_xlabel('x')
ax.set_ylabel('エンコード次元')
plt.colorbar(im, ax=ax, label='値')

plt.tight_layout()
plt.show()

print("観察:")
print("- 低周波成分（i=0）: ゆっくり変化 → 大まかな位置")
print("- 高周波成分（i=5）: 急速に変化 → 細かい位置の違い")
print("- 各位置が固有の'指紋'を持つ → ニューラルネットワークが位置を区別しやすい")

---

## 7. まとめとセルフチェック

### 学習内容の要約

| トピック | 重要なポイント |
|---------|---------------|
| Ray Casting | ピクセルからレイを生成し、シーンと交差を計算 |
| レイの表現 | $\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}$ （原点 + 距離 × 方向）|
| 座標変換 | ピクセル → カメラ座標 → ワールド座標 |
| サンプリング | 均一、層化、階層的（NeRFスタイル）|
| 交差判定 | 球、平面、AABBとの交差計算 |
| NeRFとの関連 | レイ生成 → サンプリング → NN クエリ → レンダリング |

In [ ]:
# セルフチェッククイズ
print("="*60)
print("セルフチェッククイズ")
print("="*60)

questions = [
    {
        "q": "Q1: レイの方程式 r(t) = o + td で、t が表すものは？",
        "options": ["a) 時間", "b) 温度", "c) 原点からの距離", "d) 色"],
        "answer": "c",
        "explanation": "t はレイ原点からの距離パラメータです。t=0 で原点、t>0 でレイに沿った点を表します。"
    },
    {
        "q": "Q2: ピクセル座標からカメラ座標へのレイ方向を求めるのに必要な行列は？",
        "options": ["a) 回転行列 R", "b) 内部パラメータ K の逆行列", "c) 外部パラメータ [R|t]", "d) ホモグラフィ H"],
        "answer": "b",
        "explanation": "K⁻¹ を使ってピクセル座標を正規化カメラ座標に変換します。"
    },
    {
        "q": "Q3: NeRFで層化サンプリングを使う主な理由は？",
        "options": ["a) 計算速度の向上", "b) メモリ削減", "c) エイリアシングの軽減", "d) ネットワークの簡略化"],
        "answer": "c",
        "explanation": "層化サンプリングは各区間内でランダムにサンプルすることで、均一サンプリングによるエイリアシング（縞模様など）を軽減します。"
    },
    {
        "q": "Q4: レイと球の交差判定で得られる方程式の種類は？",
        "options": ["a) 一次方程式", "b) 二次方程式", "c) 三次方程式", "d) 超越方程式"],
        "answer": "b",
        "explanation": "球の方程式にレイを代入すると at² + bt + c = 0 の二次方程式になります。"
    },
    {
        "q": "Q5: NeRFで位置エンコーディングを使う理由は？",
        "options": ["a) 次元削減", "b) ノイズ除去", "c) 高周波の詳細を学習可能にする", "d) 計算の並列化"],
        "answer": "c",
        "explanation": "MLPは低周波バイアスがあり、そのままでは高周波の詳細を学習しにくいです。位置エンコーディングにより高周波成分を明示的に与えます。"
    }
]

for i, q in enumerate(questions):
    print(f"\n{q['q']}")
    for opt in q['options']:
        print(f"  {opt}")

print("\n" + "="*60)
print("解答は下のセルを実行してください")
print("="*60)

In [ ]:
# 解答表示
print("解答と解説:")
print("="*60)
for i, q in enumerate(questions):
    print(f"\nQ{i+1}: 正解は {q['answer']}")
    print(f"   {q['explanation']}")

### 次のステップ

このNotebookでRay Castingと座標系の基礎を学びました。次のNotebookでは：

- **62. ボリュームレンダリング**: レイに沿った色と密度の統合方法
- **63. NeRF入門**: 実際のNeRFの仕組みと実装

---

**ナビゲーション**

[← 60. バンドル調整](./60_bundle_adjustment_v1.ipynb) | [62. ボリュームレンダリング →](./62_volume_rendering_v1.ipynb)